# Введение в машинное обучение для Java-разработчиков
### Практическое задание 2.  Деревья решений, случайный лес.
### Дата выдачи: 26.10.2023

### Дедлайн: 23:59MSK 08.11.2023

## О задании
В этом задании мы попытаемся разобраться в устройстве деревьев решений и ансамбля на основе деревьев. 

## Оценивание и штрафы
Каждая из задач (помечены тегом [task]) имеет определенное количество баллов (указана в скобках около задачи). Максимально допустимая оценка за работу — 15 баллов. 

- от 4 до 8 баллов - оценка "3"
- от 8 до 14 баллов - оценка "4"
- 15 баллов - оценка "5"

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов, что автоматически ведет к несдаче курса. Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в комментариях. 
В данном задании есть необязательные бонусные задания, выполнение которых добавляет баллы в карму :)

## Формат сдачи
Задания сдаются путем форка основного репозитория, коммита решения в мастер-ветку вашего форка и оповещении преподавателя о выполнении ДЗ. 

Для удобства проверки самостоятельно посчитайте свою максимальную оценку (исходя из набора решенных задач) и укажите ниже.

[[Укажите количество набранных баллов]]

## Часть 1. Дерево решений

- [task] Реализуйте подбор признака и порога для поиска условия разбиения выборки методом перебора (2 балла)
- [task] Реализуйте метод predict для инференса дерева на датасете (1 балл)
- [task] Обучите дерево на датасете ирисов Фишера (1 балл)
- Бонусное задание: В методе `_calculate_leaf_value` возвращайте вероятность классов
- Бонусное задание: Замените критерий Джини на энтропийный критерий
4 балла

In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

class DecisionNode:
    def __init__(self, feature=None, threshold=None, value=None, true_branch=None, false_branch=None):
        self.feature = feature
        self.threshold = threshold
        self.value = value
        self.true_branch = true_branch
        self.false_branch = false_branch

class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=5):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None

    def fit(self, X, y):
        self.root = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        if depth == self.max_depth or len(y) < self.min_samples_split:
            return DecisionNode(value=self._calculate_leaf_value(y))

        best_feature, best_threshold = self._find_best_split(X, y)
        if best_feature is None or best_threshold is None:
            return DecisionNode(value=self._calculate_leaf_value(y))

        mask = X[:, best_feature] <= best_threshold
        true_branch = self._build_tree(X[mask], y[mask], depth + 1)
        false_branch = self._build_tree(X[~mask], y[~mask], depth + 1)

        return DecisionNode(feature=best_feature, threshold=best_threshold, true_branch=true_branch, false_branch=false_branch)

    def _find_best_split(self, X, y):
        best_gain = 0
        best_feature = None
        best_threshold = None
        n_features = X.shape[1]
        
        for i in range(n_features):
            curXColumn = X[:, i]
            cur_gain, cur_threshold =  self._find_best_split_on_column(curXColumn, y)

            if cur_gain > best_gain:
                best_gain = cur_gain
                best_feature = i
                best_threshold = cur_threshold
            
        return best_feature, best_threshold

    def _split_y(self, x_column, y_values, x_split_value):
        y_true = [y for idx, element in enumerate(x_column) if element <= x_split_value for y in [y_values[idx]]]
        y_false = [y for idx, element in enumerate(x_column) if element > x_split_value for y in [y_values[idx]]]
        return y_true, y_false

    def _find_best_split_on_column(self, x_column, y_values):
        column_best_gain = 0
        column_best_threshold = None
        for idx, element in enumerate(x_column):
            y_true, y_false = self._split_y(x_column, y_values, element)
            cur_gain = self._calculate_gain(y_values, y_true ,y_false)
            if column_best_gain < cur_gain:
                column_best_gain = cur_gain
                column_best_threshold = element
        return column_best_gain, column_best_threshold

    def _calculate_gain(self, y, y_true, y_false):
        p = len(y_true) / len(y)
        impurity_before = self._gini_impurity(y)
        impurity_after = p * self._gini_impurity(y_true) + (1 - p) * self._gini_impurity(y_false)
        return impurity_before - impurity_after

    def _gini_impurity(self, y):
        classes, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return 1 - np.sum(probabilities ** 2)

    def _calculate_leaf_value(self, y):
        classes, counts = np.unique(y, return_counts=True)
        most_common_class = classes[np.argmax(counts)]
        return most_common_class

    def predict(self, X):
        predictions = []

        for arr in X:
            predictions.append(self._next_node(self.root, arr))
        
        return predictions

    def _next_node(self, node, x):
        if node.value is not None:
            return node.value
            
        if x[node.feature] <= node.threshold:
            return self._next_node(node.true_branch, x)
        else:
            return self._next_node(node.false_branch, x)
        
        
def print_tree(node, depth=0):
    indent = "    " * depth
    if node.value is not None:
        print(indent + "Predicted value:", node.value)
    else:
        print(indent + "Feature", node.feature, "<=", node.threshold)
        print(indent + "--> True:")
        print_tree(node.true_branch, depth + 1)
        print(indent + "--> False:")
        print_tree(node.false_branch, depth + 1)

In [2]:
data = load_iris(as_frame=True)
X = data["data"].values
y = data["target"].values

tree = DecisionTree()
tree.fit(X, y)
y_new = tree.predict(X)

print('y_new  y')
for i in range(len(y)):
    print(y_new[i], y[i], sep='      ')

y_new  y
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
0      0
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
1      1
2      2
2      2
2      2
2      2
2      2
2      2
2      2
2      2
2      2
2      2
2

### Часть 2. Случайный лес

##### [task] 1. Усреднение классификаторов (2 балла)
Реализуйте бэггинг над решающими деревьями (усреднение предсказанных вероятностей всего ансамбля или путем "голосования"). В качестве базового алгоритма используйте либо наш класс или DecisionTreeClassifier из пакета scikit-learn. 

##### [task] 2. Бутстрап обучающей выборки (2 балла)
Добавим к нашему усреднению предсказаний бутстрап выборки (генерация случайной выборки того же размера с возвращением). Сгенерируйте с помощью него отдельную обучающую выборку для каждого дерева, обучите их и усредните предсказания, как в предыдущем пункте.

##### [task] 3. Выбор случайного подмножества признаков при построении нового дерева. (2 балла)
Обучайте каждое дерево на отдельной бутстрап-выборке и случайно выбирайте признаки для обучения. 

##### [task] 4. Подсчитайте метрики оценки качества классификации (accuracy, precision, recall, ROC AUC, F-мера) для каждого из вариантов дерева. Сделайте отдельную функцию для подсчета метрик (1 балл)

- Используемый датасет https://archive.ics.uci.edu/dataset/94/spambase
- Все гиперпараметры необходимо вынести аргументы соответсвующей функции.

In [3]:
# fetch dataset 
!pip install ucimlrepo

from ucimlrepo import fetch_ucirepo 
spambase = fetch_ucirepo(id=94) 
  
# data (as numpy arrays) 
X = spambase.data.features.values
y = spambase.data.targets.values
  
# metadata 
print(spambase.metadata) 
  
# variable information 
print(spambase.variables)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

{'uci_id': 94, 'name': 'Spambase', 'repository_url': 'https://archive.ics.uci.edu/dataset/94/spambase', 'data_url': 'https://archive.ics.uci.edu/static/public/94/data.csv', 'abstract': 'Classifying Email as Spam or Non-Spam', 'area': 'Computer Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 4601, 'num_features': 57, 'feature_types': ['Integer', 'Real'], 'demographics': [], 'target_col': ['Class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1999, 'last_updated': 'Mon Aug 28 2023', 'dataset_doi': '10.24432/C53G6X', 'creators': ['Mark Hopkins', 'Erik Reeber', 'George Forman', 'Jaap Suermondt'], 'intro_paper': None, 'additional_info': {'summary': 'The "spam" concept is diverse: advertisements for products/web sites, make money fast schemes, chain letters, pornography...\n\nThe classification task for this dataset is to determine whether a given email is spam or not.\n\t\nOur collecti

In [12]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

base_classifier = DecisionTreeClassifier()

# Создаем классификатор с использованием бэггинга и обучаем модель
bagging_classifier = BaggingClassifier(base_classifier)
bagging_classifier.fit(X_train, y_train)

# Делаем предсказания на тестовой выборке
y_pred = bagging_classifier.predict(X_test)

for i in range(len(y_test)):
    print(y_pred[i], y_test[i], sep='      ')

/home/pavel/miniconda3/envs/introToML/lib/python3.11/site-packages/sklearn/ensemble/_bagging.py:802: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0      [0]
0      [0]
0      [0]
1      [1]
0      [0]
0      [1]
0      [0]
0      [0]
0      [0]
0      [0]
0      [0]
0      [0]
0      [0]
1      [1]
0      [1]
0      [0]
1      [1]
1      [1]
1      [1]
0      [0]
0      [0]
1      [1]
1      [1]
0      [0]
1      [1]
0      [0]
1      [1]
0      [0]
1      [1]
1      [1]
1      [1]
1      [1]
0      [0]
0      [0]
0      [0]
0      [0]
1      [1]
1      [1]
0      [0]
1      [1]
1      [1]
0      [0]
0      [0]
0      [0]
1      [1]
1      [1]
1      [1]
0      [0]
0      [1]
1      [1]
1      [1]
0      [0]
0      [0]
1      [1]
0      [0]
0      [0]
0      [0]
0      [0]
0      [0]
0      [0]
1      [1]
0      [0]
1      [1]
1      [1]
1      [1]
0      [0]
0      [0]
0      [0]
1      [1]
0      [0]
0      [0]
0      [0]
0      [0]
1      [1]
1      [1]
1      [0]
0      [0]
0      [0]
0      [0]
1      [1]
0      [0]
0      [0]
0      [0]
0      [0]
0      [0]
0      [0]
1      [1]
1      [1]
0      [1]
0      [0]
1      [1]

In [17]:
from sklearn.utils import resample
from sklearn.tree import DecisionTreeRegressor

def bootstrap_sample(X, y):
    indices = resample(range(len(X)), replace=True)
    return X[indices], y[indices]

# Количество деревьев
n_estimators = 100
# Список для хранения предсказаний каждого дерева
predictions = []

# Генерируем отдельную обучающую выборку и обучаем каждое дерево
for i in range(n_estimators):
    # Генерируем бутстрап выборку
    X_boot, y_boot = bootstrap_sample(X_train, y_train)
    
    model = DecisionTreeRegressor()
    # Обучаем модель на бутстрап выборке
    model.fit(X_boot, y_boot)
    # Делаем предсказание на тестовой выборке
    y_pred = model.predict(X_test)
    # Сохраняем предсказания
    predictions.append(y_pred)

# Усредняем предсказания всех деревьев
ensemble_prediction = np.mean(predictions, axis=0)

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

def calculate_metrics(true_labels, predicted_labels):
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)
    roc_auc = roc_auc_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels)

    return accuracy, precision, recall, roc_auc, f1

trees = []
num_trees = 100

# Размер бутстрап-выборки (размер выборки с возвращением)
bootstrap_size = int(0.8 * len(X_train))

# Число случайно выбранных признаков для обучения каждого дерева
num_features = int(np.sqrt(len(X_train[0])))



# Обучение каждого дерева в случайном лесу
for _ in range(num_trees):
    # Создание бутстрап-выборки
    bootstrap_indices = random.choices(range(len(X_train)), k=bootstrap_size)
    X_bootstrap = X_train[bootstrap_indices]
    y_bootstrap = y_train[bootstrap_indices]
    
    # Создание случайно выбранных признаков
    random_features = random.sample(range(len(X_train[0])), num_features)
    X_bootstrap = X_bootstrap[:, random_features]
    
    # Обучение дерева
    tree = RandomForestClassifier()
    tree.fit(X_bootstrap, y_bootstrap)
    
    # Добавление обученного дерева в список
    trees.append(tree)

    new_predict = tree.predict(X_test)
    accuracy, precision, recall, roc_auc, f1 = calculate_metrics(y_test, new_predict)
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("ROC AUC:", roc_auc)
    print("F1-score:", f1)



/home/pavel/miniconda3/envs/introToML/lib/python3.11/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


ValueError: X has 57 features, but RandomForestClassifier is expecting 7 features as input.

#### Часть 3. 

- [task] Обучите RandomForestClassifier из sklearn на датасете из прошлой части (2 балла)
- [task] Подсчитайте accuracy, precision, recall, ROC AUC, F-мера на отложенной выборке. Получилось лучше или хуже по сравнению с вашим вариантом RandomForest (2 балла)
- Попробуйте объяснить результат.

In [22]:
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)

my_predicts = classifier.predict(X_test)
print("Accuracy:", accuracy_score(y_test, my_predicts))
print("Recall:", recall_score(y_test, my_predicts))
print("Precision:", precision_score(y_test, my_predicts))
print("F1:", f1_score(y_test, my_predicts))
print("ROC AUC", roc_auc_score(true_labels, predicted_labels))



/home/pavel/miniconda3/envs/introToML/lib/python3.11/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Accuracy: 0.9522258414766558
Recall: 0.9128205128205128
Precision: 0.9726775956284153
F1: 0.9417989417989417
